# PsiCI: effect of the configuration basis on CI results

We illustrate the effect of using different configuration bases in configuration-interaction (CI) calculations with the `PsiCI` module.

In [1]:
import psi4
import numpy as np
import psiCI

# Molecular model

As usual, A preliminary step for the CI calculation is to define the molecular model and `psi4` claculation parameters.

In [2]:
# Molecular model
mol = psi4.geometry("""
         0 1
         H   0.865 0 0
         Li -0.865 0 0
         symmetry c1 """)

# psi4 parameters
psi4.set_options({
        'basis':        'cc-pvtz',
        'scf_type':     'pk',
        'reference':    'rhf',
        'mp2_type':     'conv',
        'e_convergence': 1e-9,
        'd_convergence': 1e-9})

# CI with HF orbitals

In this section, we look at the results of CI calculations using Hartree-Fock orbitals for the configuration-state basis. We start by calculating the HF ground-state with `psi4`

In [3]:
# HF ground state
scf_e, wfn = psi4.energy('scf', return_wfn=True)

# Print out the orbital energies
E = psiCI.au2ev(wfn.epsilon_a().np)
E = E[E < 5]
print("HF orbitals energies (eV)")
for k in range(len(E)):
    print("  > orbital " + str(k+1) + " = " + str(E[k]))

HF orbitals energies (eV)
  > orbital 1 = -66.8973411684478
  > orbital 2 = -7.995883660420343
  > orbital 3 = -0.07896307527438816
  > orbital 4 = 0.916776346861161
  > orbital 5 = 0.916776346861216
  > orbital 6 = 2.0416386869900314
  > orbital 7 = 3.4407687471187827
  > orbital 8 = 3.440768747118819
  > orbital 9 = 3.4889251902654124
  > orbital 10 = 4.962544677204223


We limit our active CI space to orbitals with energy smaller than 5 eV and set a full CI with single excitations (CIS) to those orbitals

In [4]:
# Create the psiCI object
PCI = psiCI.psiCI(wfn)

# Set configuration state basis
MO = np.asarray(range(1,len(E)+1));
SO = np.concatenate((-MO,MO))

PCI.setConfiguration(mode="CIS",active=SO)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.002
  * Last modified: 03/04/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Excitations = [ 3  4  5  6  7  8  9 10]
                  [ -3  -4  -5  -6  -7  -8  -9 -10]
  * 33 configurations

################################################################################



We can now perform the CI-matrix calculation and display the resulting ground state

In [5]:
CI = PCI.computeCI()
PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.002
  * Last modified: 03/04/2025

=== Electronic structure =======================================================
  * 4 electrons
  * 44 spatial orbitals
  * 33 configurations

=== Configuration-interaction calculation ======================================
  * tolerance: 1e-10
  * 10 active spatial orbitals

=== CI matrix calculation ======================================================
  * Core Hamiltonian                                                        done
  * Two-electron integrals                                                  done
  * CI matrix                                                               done

###################################################

We know that this is the same as the HF ground state (indeed, the CIS ground state is composed of 100% reference). To improve upon this, we expand the configuration-state basis to include single and double excitations (CISD), keeping the same active space of HF orbitals with less than 5 eV energy.

In [6]:
# Update configuration state basis
PCI.setConfiguration(mode="CISD",active=SO)

# Recalculate the CI matrix and display the new ground state
CI = PCI.computeCI()
PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.002
  * Last modified: 03/04/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Excitations = [ 3  4  5  6  7  8  9 10]
                  [ -3  -4  -5  -6  -7  -8  -9 -10]
  * 345 configurations

################################################################################

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Vers

With adding the double excitations, the ground state gets about 0.5% population from configuration states other than the reference and its energy is loweved by 0.05 eV.

# CI with DFT orbitals

In this section we repeat the analysis of the previous section using density-functional theory (DFT) orbitals instead of the HF ones. For that, we update the `psi4` parameters to DFT and recalculate the ground-state orbitals

In [7]:
# HF ground state
scf_e, wfn = psi4.energy('b3lyp', return_wfn=True)

# Print out the orbital energies
E = psiCI.au2ev(wfn.epsilon_a().np)
E = E[E < 5]
print("HF orbitals energies (eV)")
for k in range(len(E)):
    print("  > orbital " + str(k+1) + " = " + str(E[k]))

HF orbitals energies (eV)
  > orbital 1 = -54.77445559803659
  > orbital 2 = -5.205345261235375
  > orbital 3 = -1.4798536116209933
  > orbital 4 = -0.2590248127422943
  > orbital 5 = -0.2590248127419164
  > orbital 6 = 1.063699546267322
  > orbital 7 = 1.8121157621780182
  > orbital 8 = 1.8121157621790105
  > orbital 9 = 1.8986050827021106
  > orbital 10 = 3.1727553156394377
  > orbital 11 = 3.9362261441563486
  > orbital 12 = 3.936226144156379
  > orbital 13 = 4.015415574226257
  > orbital 14 = 4.015416330217502


Next, we perform a CIS calculation with an active space including all orbitals with energy smaller than 5 eV (note that we have more orbitals than in the HF case)

In [8]:
# Create the psiCI object
PCI = psiCI.psiCI(wfn)

# Set configuration state basis
MO = np.asarray(range(1,len(E)+1));
SO = np.concatenate((-MO,MO))

PCI.setConfiguration(mode="CIS",active=SO)

# Compute the CIS matrix and display results
CI = PCI.computeCI()

PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.002
  * Last modified: 03/04/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Excitations = [ 3  4  5  6  7  8  9 10 11 12 13 14]
                  [ -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14]
  * 49 configurations

################################################################################

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Vis

The DFT-orbital ground state wave function is almost exclusively single determinantal, with less than 1% population from single-excitation configuration states. The corresponding energy is slightly higher than with HF orbitals, indicating that in this case, we are probably not improving on the HF ground state.

In [9]:
# Update configuration state basis
PCI.setConfiguration(mode="CISD",active=SO)

# Recalculate the CI matrix and display the new ground state
CI = PCI.computeCI()
PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.002
  * Last modified: 03/04/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Excitations = [ 3  4  5  6  7  8  9 10 11 12 13 14]
                  [ -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14]
  * 757 configurations

################################################################################

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. V

We now have the ground-state wave function composed of slightly over 1% population from configuration states other than the reference and the energy is 0.09 eV lower than that of HF. 